In [3]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import json
from dotenv import load_dotenv # 需要一个.env file
import os

In [6]:
load_dotenv()

JSON_INDENT = 2
INDEX_NAME = "students-sudo" #数据库名称

client = Elasticsearch(
    "https://127.0.0.1:9200/students",
    verify_certs=False, 
    basic_auth=("elastic", os.getenv("ELASTIC_PASSWORD"))# 在cluster之后就要改
)

c:\Users\91891\AppData\Local\Programs\Python\Python38\lib\site-packages\elasticsearch\_sync\client\__init__.py:399: SecurityWarning: Connecting to 'https://127.0.0.1:9200/students' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [8]:
print(client.info()) #看下有没有连接成功

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPSConnection object at 0x000001D4EA8E0BE0>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。))

In [ ]:
# DEFINE THE setting and mappings for the new index
def create_student_index():
    index_body = {
        "settings": {
            "index": {
                "number_of_shards": 3,
                "number_of_replicas": 1
            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "name": {"type": "text"},
                "course": {"type": "text"},
                "mark": {"type": "short"}
            }
        }
    }
    response = client.indices.create(index = INDEX_NAME, body = index_body)
    return response


In [ ]:
# 批量地Insert document 
def insert_student_data():
    student_data = [
        {
            "_index":INDEX_NAME,
            "_id":"123123", 
            "_source":{
                "name":"SUDO student1", 
                "crouse": "sadasd", 
                "mark":80
            }
            
        }, {
            "_index":INDEX_NAME,
            "_id":"145413", 
            "_source":{
                "name":"SUDO student2", 
                "crouse": "sadasd", 
                "mark":80
            }
            
        }


    ]

    response = bulk(client, student_data)

In [ ]:
# 查
def get_ccc_student():
    response = client.search(
        index = INDEX_NAME, 
        body={
            "query":{
                "bool":{
                    "must":[
                        {"range":{"mark":{"gt":85}}}, 
                        {
                            "has_parent":{
                                "parent_type":"course", 
                                "query":{"match":{"coursedesscription":"cloud"}}
                            }
                        }
                    ]
                }
            }
        }
    )
    return response

# 从get到的字典里面提取需要的信息
def print_formmarred_response(res):
    print(json.dumps(res['hits']["hits"], indent = JSON_INDENT))

print_formmarred_response(get_ccc_student())

In [ ]:
def get_h1_student():
    response = client.search(
        index = INDEX_NAME, 
        body={
            "query":{
                "bool":{
                    "must":[
                        {"match":{
                    "course" : "ccc" # 这里是关键字搜索， 只用包含就行
                    }}
                    ], 
                    "filter":[
                        {"range":{"mark":{"gt":85}}}
                    ]
                }
                # 找到所有course = ccc的
                
            }
        }
    )
    return response  
    

In [ ]:
# print(create_student_index())
# print(insert_student_data())
# print_formmarred_response(get_ccc_student)
# print_formmarred_response(get_h1_student)